PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.1 py4j==0.10.9.7 

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Titanic Binary Classification example") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/26 14:35:54 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/26 14:35:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 14:35:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 타이타닉 생존 예측 모델 만들기




In [3]:
spark

In [4]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv

--2024-06-26 14:36:05--  https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.133.34, 3.5.82.188, 3.5.76.161, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.133.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61197 (60K) [text/csv]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  59.76K   359KB/s    in 0.2s    

2024-06-26 14:36:05 (359 KB/s) - ‘titanic.csv’ saved [61197/61197]



In [5]:
data = spark.read.csv('./titanic.csv', header=True, inferSchema=True)

In [9]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [10]:
data.select(['*']).describe().show()

24/06/26 14:38:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|Gender|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                NULL|  NULL| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

**데이터 클린업**: 

*   PassengerID, Name, Ticket, Embarked는 사용하지 않을 예정 (아무 의미가 없음).
*   Cabin도 비어있는 값이 너무 많아서 사용하지 않을 예정
*   Age는 중요한 정보인데 비어있는 레코드들이 많아서 디폴트값을 채워줄 예정
*   Gender의 경우 카테고리 정보이기에 숫자로 인코딩 필요



In [11]:
final_data = data.select(['Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare'])

In [12]:
final_data.show()

+--------+------+------+----+-----+-----+-------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|
+--------+------+------+----+-----+-----+-------+
|       0|     3|  male|22.0|    1|    0|   7.25|
|       1|     1|female|38.0|    1|    0|71.2833|
|       1|     3|female|26.0|    0|    0|  7.925|
|       1|     1|female|35.0|    1|    0|   53.1|
|       0|     3|  male|35.0|    0|    0|   8.05|
|       0|     3|  male|NULL|    0|    0| 8.4583|
|       0|     1|  male|54.0|    0|    0|51.8625|
|       0|     3|  male| 2.0|    3|    1| 21.075|
|       1|     3|female|27.0|    0|    2|11.1333|
|       1|     2|female|14.0|    1|    0|30.0708|
|       1|     3|female| 4.0|    1|    1|   16.7|
|       1|     1|female|58.0|    0|    0|  26.55|
|       0|     3|  male|20.0|    0|    0|   8.05|
|       0|     3|  male|39.0|    1|    5| 31.275|
|       0|     3|female|14.0|    0|    0| 7.8542|
|       1|     2|female|55.0|    0|    0|   16.0|
|       0|     3|  male| 2.0|    4|    1| 29.125|


Age는 평균값으로 채운다

In [13]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])
imputer_model = imputer.fit(final_data)
final_data = imputer_model.transform(final_data)

In [14]:
final_data.select("Age", "AgeImputed").show()

+----+-----------------+
| Age|       AgeImputed|
+----+-----------------+
|22.0|             22.0|
|38.0|             38.0|
|26.0|             26.0|
|35.0|             35.0|
|35.0|             35.0|
|NULL|29.69911764705882|
|54.0|             54.0|
| 2.0|              2.0|
|27.0|             27.0|
|14.0|             14.0|
| 4.0|              4.0|
|58.0|             58.0|
|20.0|             20.0|
|39.0|             39.0|
|14.0|             14.0|
|55.0|             55.0|
| 2.0|              2.0|
|NULL|29.69911764705882|
|31.0|             31.0|
|NULL|29.69911764705882|
+----+-----------------+
only showing top 20 rows



In [15]:
final_data.show()

+--------+------+------+----+-----+-----+-------+-----------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|
+--------+------+------+----+-----+-----+-------+-----------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|
|       0|     3|  male|NULL|    0|    0| 8.4583|29.69911764705882|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|
|       0|     3|  male| 2.0|    3|    1| 21.075|              2.0|
|       1|     3|female|27.0|    0|    2|11.1333|             27.0|
|       1|     2|female|14.0|    1|    0|30.0708|             14.0|
|       1|     3|female| 4.0|    1|    1|   16.7|              4.0|
|       1|     1|female|58.0|    0|    0|  26.55

성별 정보 인코딩: male -> 0, female -> 1

In [16]:
from pyspark.ml.feature import StringIndexer

gender_indexer = StringIndexer(inputCol='Gender', outputCol='GenderIndexed')
gender_indexer_model = gender_indexer.fit(final_data)
final_data = gender_indexer_model.transform(final_data)

In [17]:
final_data.select("Gender", "GenderIndexed").show()

+------+-------------+
|Gender|GenderIndexed|
+------+-------------+
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
|  male|          0.0|
|  male|          0.0|
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
|  male|          0.0|
|female|          1.0|
|female|          1.0|
+------+-------------+
only showing top 20 rows



In [18]:
final_data.show()

+--------+------+------+----+-----+-----+-------+-----------------+-------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|GenderIndexed|
+--------+------+------+----+-----+-----+-------+-----------------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|          0.0|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|          1.0|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|          1.0|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|          1.0|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|          0.0|
|       0|     3|  male|NULL|    0|    0| 8.4583|29.69911764705882|          0.0|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|          0.0|
|       0|     3|  male| 2.0|    3|    1| 21.075|              2.0|          0.0|
|       1|     3|female|27.0|    0|    2|11.1333|             27.0|          1.0|
|       1|     2

## 피쳐 벡터를 만들기

In [19]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeImputed', 'GenderIndexed'], outputCol='features')
data_vec = assembler.transform(final_data)

In [20]:
data_vec.show()

+--------+------+------+----+-----+-----+-------+-----------------+-------------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|GenderIndexed|            features|
+--------+------+------+----+-----+-----+-------+-----------------+-------------+--------------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|          0.0|[3.0,1.0,0.0,7.25...|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|          1.0|[1.0,1.0,0.0,71.2...|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|          1.0|[3.0,0.0,0.0,7.92...|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|          1.0|[1.0,1.0,0.0,53.1...|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|          0.0|[3.0,0.0,0.0,8.05...|
|       0|     3|  male|NULL|    0|    0| 8.4583|29.69911764705882|          0.0|[3.0,0.0,0.0,8.45...|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|      

## 훈련용과 테스트용 데이터를 나누고 binary classification 모델을 하나 만든다

In [21]:
train, test = data_vec.randomSplit([0.7, 0.3])

In [22]:
from pyspark.ml.classification import LogisticRegression

algo = LogisticRegression(featuresCol="features", labelCol="Survived")
model = algo.fit(train)

24/06/26 14:45:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


## 모델 성능 측정

In [23]:
predictions = model.transform(test)

In [24]:
predictions.select(['Survived','prediction', 'probability']).show()

+--------+----------+--------------------+
|Survived|prediction|         probability|
+--------+----------+--------------------+
|       0|       0.0|[0.53429912620917...|
|       0|       0.0|[0.51292755659774...|
|       0|       0.0|[0.50933874009195...|
|       0|       0.0|[0.50240064727804...|
|       0|       1.0|[0.49400940943235...|
|       0|       1.0|[0.35504674600719...|
|       0|       1.0|[0.43498883989822...|
|       0|       1.0|[0.48977782293703...|
|       0|       1.0|[0.40033477257721...|
|       0|       1.0|[0.35150456065382...|
|       0|       0.0|[0.57954038489510...|
|       0|       0.0|[0.61630426655736...|
|       0|       0.0|[0.68051169506838...|
|       0|       0.0|[0.67138348689075...|
|       0|       0.0|[0.74063133966651...|
|       0|       0.0|[0.71780382149804...|
|       0|       0.0|[0.75358697899261...|
|       0|       0.0|[0.73888276510766...|
|       0|       0.0|[0.78244577400387...|
|       0|       0.0|[0.79381521185513...|
+--------+-

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='Survived', metricName='areaUnderROC')
evaluator.evaluate(predictions)

0.8541435628591602